In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import pandas as pd
from sklearn.model_selection import train_test_split
import pandas as pd
from ucimlrepo import fetch_ucirepo
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# preprocessing

diabetes_130_us_hospitals_for_years_1999_2008 = fetch_ucirepo(id=296)
X = diabetes_130_us_hospitals_for_years_1999_2008.data.features
y = diabetes_130_us_hospitals_for_years_1999_2008.data.targets

columns_to_drop = ['weight', 'payer_code', 'medical_specialty', 'max_glu_serum', 'A1Cresult']

# drop features: weight, payer_code, medical_specialty, max_glu_serum, A1Cresult
X = X.drop(columns=columns_to_drop)
X.drop(X[X['gender'] == 'Unknown/Invalid'].index, inplace=True)
# results in 105 column features after transform

# labels = ['Circulatory', 'Respiratory', 'Digestive', 'Diabetes', 'Injury',
#           'Musculoskeletal', 'Genitourinary', 'Neoplasms', 'Other']
def map_icd_to_category(icd_code):
    try:
        icd_code = float(icd_code)
    except ValueError:
        return "Other"
    if 390 <= icd_code <= 459 or icd_code == 785:
        return 'Circulatory'
    elif 460 <= icd_code <= 519 or icd_code == 786:
        return 'Respiratory'
    elif 520 <= icd_code <= 579 or icd_code == 787:
        return 'Digestive'
    elif str(icd_code).startswith('250'):
        return 'Diabetes'
    elif 800 <= icd_code <= 999:
        return 'Injury'
    # Musculoskeletal
    elif 710 <= icd_code <= 739:
        return 'Musculoskeletal'
    # Genitourinary
    elif 580 <= icd_code <= 629 or icd_code == 788:
        return 'Genitourinary'
    # Neoplasms
    elif 140 <= icd_code <= 239:
        return 'Neoplasms'
    else:
        return 'Other'

X['diag_1'] = X['diag_1'].apply(map_icd_to_category)
X['diag_2'] = X['diag_2'].apply(map_icd_to_category)
X['diag_3'] = X['diag_3'].apply(map_icd_to_category)

categorical_columns = X.select_dtypes(include=['object']).columns
encoder = OneHotEncoder(sparse=False)

# Fit and transform the categorical columns
X_encoded = encoder.fit_transform(X[categorical_columns])
X_encoded = pd.DataFrame(X_encoded, columns=encoder.get_feature_names_out(categorical_columns))
X_encoded = pd.concat([X_encoded, X.drop(columns=categorical_columns)], axis=1)

column_variances = X_encoded.var()
nzv = set()
# set of near zero variance features
for column, variance in column_variances.items():
    if variance < .001:
        nzv.add(column.split('_')[0])

columns_to_drop = []
for column in X_encoded.columns:
    parts = column.split('_')
    if parts[0] in nzv:
        columns_to_drop.append(column)

X_encoded = X_encoded.drop(columns=columns_to_drop)

missing_values = X_encoded.isnull().any(axis=1)
X_encoded = X_encoded[~missing_values]
y = y[~missing_values]

y['readmitted'] = y['readmitted'].map({'<30': 1, '>30': 1, 'NO': 0})

# pd.set_option('display.max_columns', None)
# print(X_encoded.head(1000))


X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [ ]:
mlp_model = MLPClassifier(max_iter = 300)
parameter_space = {
    'hidden_layer_sizes': [(64,), (64, 64, 64), (64, 64, 64, 64, 64)],
    'activation': ['relu', 'logistic', 'tanh'],
    'solver': ['adam', 'sgd'],
    'learning_rate_init': [.005, .05, .5],
    'learning_rate': ['adaptive', 'constant'],
}
mlp_model = GridSearchCV(mlp_model, parameter_space, n_jobs = -1, cv = 3)
mlp_model.best_params_

In [ ]:
mlp_model1 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64,), activation = 'logistic', learning_rate_init=.05, learning_rate= 'adaptive', solver = 'adam')
mlp_model2 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'logistic', learning_rate_init=.05, learning_rate= 'adaptive', solver = 'adam')

mlp_model3 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64,), activation = 'logistic', learning_rate_init=.05, learning_rate= 'adaptive', solver = 'sgd')
mlp_model4 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'logistic', learning_rate_init=.05, learning_rate= 'adaptive', solver = 'sgd')

mlp_model5 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'logistic', learning_rate_init=.5, learning_rate= 'adaptive', solver = 'sgd')
mlp_model6 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'logistic', learning_rate_init=.005, learning_rate= 'adaptive', solver = 'sgd')

mlp_model7 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'tanh', learning_rate_init=.05, learning_rate= 'adaptive', solver = 'sgd')
mlp_model8 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'tanh', learning_rate_init=.005, learning_rate= 'adaptive', solver = 'sgd')

mlp_model9 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'relu', learning_rate_init=.05, learning_rate= 'adaptive', solver = 'sgd')
mlp_model10 = MLPClassifier(max_iter = 300, hidden_layer_sizes = (64, 64, 64), activation = 'relu', learning_rate_init=.005, learning_rate= 'adaptive', solver = 'sgd')


In [ ]:
mlp_model1.fit(X_train, y_train)
mlp_train_acc1 = mlp_model.score(X_train, y_train)
mlp_val_acc1 = mlp_model.score(X_val, y_val)
mlp_test_acc1 = mlp_model.score(X_test, y_test)

mlp_model2.fit(X_train, y_train)
mlp_train_acc2 = mlp_model2.score(X_train, y_train)
mlp_val_acc2 = mlp_model2.score(X_val, y_val)
mlp_test_acc2 = mlp_model2.score(X_test, y_test)

mlp_model3.fit(X_train, y_train)
mlp_train_acc3 = mlp_model3.score(X_train, y_train)
mlp_val_acc3 = mlp_model3.score(X_val, y_val)
mlp_test_acc3 = mlp_model3.score(X_test, y_test)

mlp_model4.fit(X_train, y_train)
mlp_train_acc4 = mlp_model4.score(X_train, y_train)
mlp_val_acc4 = mlp_model4.score(X_val, y_val)
mlp_test_acc4 = mlp_model4.score(X_test, y_test)

mlp_model5.fit(X_train, y_train)
mlp_train_acc5 = mlp_model5.score(X_train, y_train)
mlp_val_acc5 = mlp_model5.score(X_val, y_val)
mlp_test_acc5 = mlp_model5.score(X_test, y_test)

mlp_model6.fit(X_train, y_train)
mlp_train_acc6 = mlp_model6.score(X_train, y_train)
mlp_val_acc6 = mlp_model6.score(X_val, y_val)
mlp_test_acc6 = mlp_model6.score(X_test, y_test)

mlp_model7.fit(X_train, y_train)
mlp_train_acc7 = mlp_model7.score(X_train, y_train)
mlp_val_acc7 = mlp_model7.score(X_val, y_val)
mlp_test_acc7 = mlp_model7.score(X_test, y_test)

mlp_model8.fit(X_train, y_train)
mlp_train_acc8 = mlp_model8.score(X_train, y_train)
mlp_val_acc8 = mlp_model8.score(X_val, y_val)
mlp_test_acc8 = mlp_model8.score(X_test, y_test)

mlp_model9.fit(X_train, y_train)
mlp_train_acc9 = mlp_model9.score(X_train, y_train)
mlp_val_acc9 = mlp_model9.score(X_val, y_val)
mlp_test_acc9 = mlp_model9.score(X_test, y_test)

mlp_model10.fit(X_train, y_train)
mlp_train_acc10 = mlp_model10.score(X_train, y_train)
mlp_val_acc10 = mlp_model10.score(X_val, y_val)
mlp_test_acc10 = mlp_model10.score(X_test, y_test)

In [ ]:
print("Training Accuracy Score:", mlp_train_acc1)
print("Validation Accuracy Score:", mlp_val_acc1)
print("Testing Accuracy Score:", mlp_test_acc1)

print("\nTraining Accuracy Score2:", mlp_train_acc2)
print("Validation Accuracy Score2:", mlp_val_acc2)
print("Testing Accuracy Score2:", mlp_test_acc2)

print("\nTraining Accuracy Score3:", mlp_train_acc3)
print("Validation Accuracy Score3:", mlp_val_acc3)
print("Testing Accuracy Score3:", mlp_test_acc3)

print("\nTraining Accuracy Score4:", mlp_train_acc4)
print("Validation Accuracy Score4:", mlp_val_acc4)
print("Testing Accuracy Score4:", mlp_test_acc4)

print("\nTraining Accuracy Score5:", mlp_train_acc5)
print("Validation Accuracy Score5:", mlp_val_acc5)
print("Testing Accuracy Score5:", mlp_test_acc5)

print("\nTraining Accuracy Score6:", mlp_train_acc6)
print("Validation Accuracy Score6:", mlp_val_acc6)
print("Testing Accuracy Score6:", mlp_test_acc6)

print("\nTraining Accuracy Score7:", mlp_train_acc7)
print("Validation Accuracy Score7:", mlp_val_acc7)
print("Testing Accuracy Score7:", mlp_test_acc7)

print("\nTraining Accuracy Score8:", mlp_train_acc8)
print("Validation Accuracy Score8:", mlp_val_acc8)
print("Testing Accuracy Score8:", mlp_test_acc8)

print("\nTraining Accuracy Score9:", mlp_train_acc9)
print("Validation Accuracy Score9:", mlp_val_acc9)
print("Testing Accuracy Score9:", mlp_test_acc9)

print("\nTraining Accuracy Score10:", mlp_train_acc10)
print("Validation Accuracy Score10:", mlp_val_acc10)
print("Testing Accuracy Score10:", mlp_test_acc10)

|Model Name         |Hiden Layer Sizes |Learning rate init  | Learning Rate | Activation | Solver | Validation Error
|:-----------------:|:----------------:|:------------------:|:-------------:|:----------:|:------:|:------:|
|MLP Classifer 1  |(64,)|0.05|'adaptive'|'logistic'|'adam'|0.6155|
|MLP Classifer 2  |(64,64,64)|0.05|'adaptive'|'logistic'|'adam'|0.5394|
|MLP Classifer 3  |(64,64,64)|0.05|'adaptive'|'logistic'|'sgd'|0.6320|
|MLP Classifer 4  |(64,64,64)|0.05|'adaptive'|'logistic'|'sgd'|0.6445|
|MLP Classifer 5  |(64,64,64)|0.5|'adaptive'|'logistic'|'sgd'|0.6196|
|MLP Classifer 6  |(64,64,64)|0.005|'adaptive'|'logistic'|'sgd'|0.6332|
|MLP Classifer 7  |(64,64,64)|0.05|'adaptive'|'tanh'|'sgd'|0.6197|
|MLP Classifer 8  |(64,64,64)|0.005|'adaptive'|'tanh'|'sgd'|0.6293|
|MLP Classifer 9  |(64,64,64)|0.05|'adaptive'|'relu'|'sgd'|0.6228|
|MLP Classifer 10 |(64,64,64)|0.005|'adaptive'|'relu'|'sgd'|0.6254|
|MLP Classifer 11 |(64,64,64,64,64)|0.05|'adaptive'|'logistic'|'sgd'|0.5394|
|MLP Classifer 12 |(64,64,64,64,64)|0.005|'adaptive'|'logistic'|'sgd'|0.5394|